In [1]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd

TRAIN_PATH = '/kaggle/input/tensorflow-great-barrier-reef'

### Functions

In [2]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_path(row):
    row['image_path'] = f'{TRAIN_PATH}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

np.random.seed(8)
colors = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
colors=(255,0,0)

In [3]:
# Read in the data CSV files
df = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")
df.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [4]:
df["NumBBox"]=df['annotations'].apply(lambda x: len(eval(x)))
df.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,NumBBox
0,0,40258,0,0,0-0,[],0
1,0,40258,1,1,0-1,[],0
2,0,40258,2,2,0-2,[],0
3,0,40258,3,3,0-3,[],0
4,0,40258,4,4,0-4,[],0


In [5]:
print(df["NumBBox"].unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17 15 18]


In [6]:
df_train=df[df["NumBBox"]>0]
df_train.sample(2)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,NumBBox
12642,1,60754,9077,661,1-9077,"[{'x': 506, 'y': 568, 'width': 28, 'height': 3...",7
12784,1,60754,9219,803,1-9219,"[{'x': 336, 'y': 582, 'width': 65, 'height': 5...",5


In [7]:
print(df_train['NumBBox'].sum())

11898


In [8]:
df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: eval(x))
df_train.sample(2)

  0%|          | 0/4919 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,video_id,sequence,video_frame,sequence_frame,image_id,annotations,NumBBox
11059,1,8503,5907,2079,1-5907,"[{'x': 4, 'y': 3, 'width': 65, 'height': 63}, ...",3
4769,0,53708,8946,61,0-8946,"[{'x': 975, 'y': 411, 'width': 36, 'height': 3...",2


In [9]:
df_train = df_train.progress_apply(get_path, axis=1)
df_train.sample(2)

  0%|          | 0/4919 [00:00<?, ?it/s]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,NumBBox,image_path
5530,0,53708,9707,822,0-9707,"[{'x': 496, 'y': 210, 'width': 41, 'height': 34}]",1,/kaggle/input/tensorflow-great-barrier-reef/tr...
251,0,40258,251,251,0-251,"[{'x': 897, 'y': 286, 'width': 37, 'height': 35}]",1,/kaggle/input/tensorflow-great-barrier-reef/tr...


In [10]:
from sklearn.model_selection import GroupKFold

kf = GroupKFold(n_splits = 5)
df_train = df_train.reset_index(drop=True)
df_train['fold'] = -1

for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y = df_train.video_id.tolist(), groups=df_train.sequence)):
    df_train.loc[val_idx, 'fold'] = fold

df_train.fold.value_counts()

0    1100
3     970
2     968
4     945
1     936
Name: fold, dtype: int64

In [11]:
selected_fold = 4

df_train['train'] = df_train.fold.progress_apply(lambda x: 1 if x != selected_fold else 0)

  0%|          | 0/4919 [00:00<?, ?it/s]

In [12]:
df_train.train.value_counts()

1    3974
0     945
Name: train, dtype: int64